# Modifications to the standard Gordon equilibrium solver

The Gordon equilibrium solver works well for reactions in solution or gases. Our 2 key modifications are to 

1) Modify the chemical potential equation to be zero if we run out of a solid/liquid substance (n_i = 0) and the chemical potential difference is positive ($\mu_i - \sum A \vec{\lambda} > 0$) - we have used the substance up, and there is no way to satisfy the chemical potential equation. Note that if the chemical potential difference is negative, we should instead create more of the substance.

2) Not sure if this is different than their approach, but use the exact chemical potential of each element at each iteration.

Then hopefully the same iterative procedure should work well?

In [1]:
%load_ext autoreload
%autoreload 2
import pchem.calorimetry as cal
import numpy as np
import sympy as sm

# Test case showing good performance for normal chemical equilibrium

In [2]:
S  = cal.Substance2("S(l)", state="l", H0=0, S0=0, cP=50.0, molarMass=100.0, density=1.0, atoms=dict(S=1))
P  = cal.Substance2("P(aq)", state="aq", H0=0, S0=0, cP=0, molarMass=100.0, density=1.0, atoms=dict(P=1))
D  = cal.Substance2("D(aq)", state="aq", H0=0, S0=0, cP=0, molarMass=100.0, density=1.0, atoms=dict(D=1))
PD = cal.Substance2("PD(aq)", state="aq", H0=0, S0=50.0, cP=0, molarMass=100.0, density=1.0, atoms=dict(D=1, P=1))

chemicals = {0: S, 1: P, 2: D, 3: PD}

s0 = cal.State3(T=298.15, chemicals=chemicals, rxns=[], V=1.0)

s0.set_state({0:50.0, 1: 1.0, 2: 1.0, 3: 0.0})

In [3]:
K = np.exp(50.0/8.3145)

In [4]:
x = sm.symbols('x')

x_solved = float(sm.solve(K - x/ (1-x)**2, x)[0])

In [5]:
out, iterations = s0._solve_iterative()

In [6]:
np.allclose( np.array([1-x_solved, 1-x_solved, x_solved]), out[:3])

True

## Normal chemical equilibrium: temperature change

In [7]:
out, iterations = s0._solve_iterative_dH(dH=0.0)
n_moles = out[:4]
lambdas = out[4:7]
T = out[-1]
print(f"{T=}")

T=298.15


In [8]:
out, iterations = s0._solve_iterative_dH(dH=100.0)
n_moles = out[:4]
lambdas = out[4:7]
T = out[-1]
print(f"{T=}")

cP = 2500.0
dT = 100/2500.0

np.allclose(T, s0.T + dT) # Good!

T=298.19


True

## Exothermic reaction

Here the temperature changes will be coupled to the chemical reaction.

In [9]:
S  = cal.Substance2("S(l)", state="l", H0=0, S0=0, cP=50.0, molarMass=100.0, density=1.0, atoms=dict(S=1))
P  = cal.Substance2("P(aq)", state="aq", H0=0, S0=0, cP=0, molarMass=100.0, density=1.0, atoms=dict(P=1))
D  = cal.Substance2("D(aq)", state="aq", H0=0, S0=0, cP=0, molarMass=100.0, density=1.0, atoms=dict(D=1))
PD = cal.Substance2("PD(aq)", state="aq", H0=-50*298.15, S0=0.0, cP=0, molarMass=100.0, density=1.0, atoms=dict(D=1, P=1))

chemicals = {0: S, 1: P, 2: D, 3: PD}

s0 = cal.State3(T=298.15, chemicals=chemicals, rxns=[], V=1.0)

s0.set_state({0:50.0, 1: 1.0, 2: 1.0, 3: 0.0})

In [10]:
out, iterations = s0._solve_iterative_dH(dH=0.0)
n_moles = out[:4]
lambdas = out[4:7]
T = out[-1]
print(f"{T=}")
print(f"{n_moles=}")

T=303.8253308017202
n_moles=array([4.82423609e-02, 4.82423609e-02, 9.51757639e-01, 5.00000000e+01])


In [11]:
s0.x = n_moles
s0.T = T
out, iterations = s0._solve_iterative_dH(dH=0.0)
n_moles = out[:4]
lambdas = out[4:7]
T = out[-1]
print(f"{T=}")
print(f"{n_moles=}")

T=303.8091455150239
n_moles=array([ 0.05095665,  0.05095665,  0.94904335, 50.        ])


In [12]:
s0.x = n_moles
s0.T = T
out, iterations = s0._solve_iterative_dH(dH=0.0)
n_moles = out[:4]
lambdas = out[4:7]
T = out[-1]
print(f"{T=}")
print(f"{n_moles=}")

T=303.80919202671987
n_moles=array([ 0.05094885,  0.05094885,  0.94905115, 50.        ])


In [13]:
s0.x = n_moles
s0.T = T
out, iterations = s0._solve_iterative_dH(dH=0.0)
n_moles = out[:4]
lambdas = out[4:7]
T = out[-1]
print(f"{T=}")
print(f"{n_moles=}")

T=303.8091918930699
n_moles=array([ 0.05094887,  0.05094887,  0.94905113, 50.        ])


In [14]:
iterations

[4.3867801258201666e-07, 1.2569474350243581e-09]

## Phase transition

In [25]:
S  = cal.Substance2("S(l)", state="l", H0=0, S0=0, cP=50.0, molarMass=100.0, density=1.0, atoms=dict(S=1))
Ssolid = cal.Substance2("S(s)", state="s", H0=-10000, S0=-20.0, cP=40.0, molarMass=100.0, density=1.05, atoms=dict(S=1))
chemicals = {0: Ssolid, 1: S}

s0 = cal.State3(T=298.15, chemicals=chemicals, rxns=[], V=1.0)

s0.set_state({0:10.0, 1: 0.0})

In [113]:
x0_pt = np.r_[s0.x, s0.mu()[0]/(s0.T*8.3145), s0.T]

A, y = s0._fHi(x0_pt, 400.0)

In [114]:
from scipy import optimize

In [115]:
def f(x, x0, s, eps =1e-10):
    r = A @ x - y
    x_total = (x + x0)
    for i in range(s.N_aq, s.N_chem):
        if (r[i] >=0) and (x_total[i] <= eps):
            r[i] = r[i]*x_total[i]**0.2 # You do need to have this cost present for 
            
    for i in range(s.N_chem, s.N_chem+s.N_atoms):
        r[i] = r[i] * 100 # High cost for moles errors...
    return r

In [116]:
lb = np.array([0, -10, -np.inf, -np.inf])

In [117]:
s0.chem_labels

('S(l)', 'S(s)')

In [119]:
guess = np.array([0.0, 0.0, s0.mu()[0]/(8.3145*s0.T), 0])

f(guess, x0_pt, s0)

array([ 0.        ,  0.        ,  0.        , -0.16135746])

In [121]:
out = optimize.least_squares(f, x0=np.array([0,0,s0.mu()[0]/(8.3145*s0.T),0.0]),
                       bounds=(lb, np.inf),
#                        jac=lambda x, y, z: A,
                       args=(x0_pt, s0))

In [125]:
Tf = np.exp(out.x[-1]) * x0_pt[-1]

In [126]:
Tf

299.15167888523723

In [329]:
lb = np.array([0, -10, -np.inf, -np.inf])

In [330]:
dy = A @ np.array([0,0,x0_pt[2],0])

In [331]:
m=np.array([True, True, False, True])

In [332]:
y3 = (y+dy)

In [333]:
A3 = A[:, m]

In [334]:
y3

array([-1.62850013,  0.        ,  0.        ,  0.16135746])

In [335]:
lb = np.array([0, -10, -np.inf])
y_out = optimize.lsq_linear(A3, y3, bounds=(lb, np.inf))

In [336]:
x_new = np.r_[x0_pt[:2] + y_out.x[:2], 0.0, x0_pt[-1]*np.exp(y_out.x[-1])]

In [337]:
mu2 = s0.mu(x=x_new[:2], T=x_new[-1])/(8.3145*x_new[-1])

In [338]:
mu2

array([-1.62198613])

In [344]:
A, y = s0._fHi(x_new, 2000.0)
dy = A @ np.array([0,0,mu2[0],0])
y3 = y - dy
A3 = A[:, m]

In [345]:
A

array([[ 0.        ,  0.        , -1.        , -0.        ],
       [ 0.        ,  0.        , -1.        ,  4.03393642],
       [ 1.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        , -4.03393642,  0.        , 48.15683445]])

In [352]:
U, s, Vh = linalg.svd(A)

$$A = U \sigma V^T$$

$$U^T U \sigma V^T x = y$$

$$ x = V \sigma^{-1} U^T y$$

In [364]:
Vh.T @ np.diag(s**-1) @ U.T @ y

array([-4.65934073e+00,  4.65934073e+00, -1.35185608e-16,  4.03700000e-01])

In [365]:
optimize.nonlin.l

array([-4.65934073,  4.65934073, -0.        ,  0.4037    ])

In [340]:
lb = np.r_[-x_new[:2], -np.inf]
y_out2 = optimize.lsq_linear(A3, y3, bounds=(lb, np.inf))

In [341]:
x_new2 = np.r_[x_new[:2] + y_out2.x[:2], 0.0, x_new[-1]*np.exp(y_out2.x[-1])]

In [343]:
# optimize.least_squares()

array([1.80722637e-01, 9.81927736e+00, 0.00000000e+00, 2.98631842e+02])

In [237]:
Aw = w @ A
yw = w @ y

In [238]:
lsq_lin = optimize.lsq_linear(Aw, yw, bounds=(lb, np.inf))

In [231]:
A

array([ 0.        ,  0.        , -1.62850013,  0.        ])

In [233]:
v

array([ 0.        ,  0.        , -1.62850013,  0.        ])

In [240]:
x1 = np.r_[x0_pt[:2] + lsq_lin.x[:2], lsq_lin.x[2], x0_pt[-1]*np.exp(lsq_lin.x[-1])]

In [241]:
x1

array([ 2.46833384e-01,  9.75316662e+00, -1.71199084e+00,  2.92042586e+02])

In [228]:
A @ lsq_lin.x - y

array([ 1.71199084e+00, -2.22044605e-16,  0.00000000e+00,  1.11022302e-16])

In [108]:
from scipy import linalg, optimize

In [109]:
optimize.lsq_linear()

TypeError: lstsq() missing 2 required positional arguments: 'a' and 'b'